# SST in Bavi, Mayas, Haishen
  
Authors
* [Dr Chelle Gentemann](mailto:gentemann@faralloninstitute.org)    - Farallon Institute, USA


## In Feb 2020 a GRL [paper](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020GL091430) came out connecting 3 closely occuring Typhoons near Korea to the California wildfires
  
"Strong winds that accentuated a fire outbreak in the western United States in early September of 2020 resulted from an atmospheric wave train that spanned the Pacific Ocean. Days before the atmospheric waves developed in the United States, three western Pacific tropical cyclones (typhoons) underwent an extratropical transition over Korea within an unprecedentedly short span of 12 days. "

Figure 1 showed zonal winds averaged over a box located over NCal/Oregon. On 9/5 and again on 9/24 the zonal winds were strongly negative (Diablo winds) and both events had major fires.
  

-------------------

## Import python packages

* You are going to want numpy, pandas, matplotlib.pyplot, podaaacpy, and xarray
* This cell also imports a parser so that a login file can be read to use podaacpy

In [ ]:
#!pip install xhistogram
#!pip install eofs
#!pip install lxml

import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs


import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from eofs.xarray import Eof
from eofs.examples import example_data_path

#This is for reading in and parsing the login file credentials
from pathlib import Path
import configparser
from lxml import objectify
from xhistogram.xarray import histogram
from scipy import signal

plt.rcParams["figure.figsize"] = (15,8)
#rcParams['figure.figsize'] = 5, 10


from matplotlib import pyplot as plt, animation
%matplotlib inline

# This is needed to display graphics calculated outside of jupyter notebook
from IPython.display import HTML, display


## Analysis of SSTs during Typhoons in 2020 near Korea


# Read in Storm data from a thredds server
- Note update - the thredds server has disappeared, so I have left the url in the code, but commented out and replaced it with a local copy of the data.  

In [ ]:
#url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/netcdf/IBTrACS.WP.v04r00.nc'
url = './../data/IBTrACS.WP.v04r00.nc'

ds_storm=xr.open_dataset(url)

# the data isn't formatted very well & have to convert strings
for var in ds_storm:
    if not((ds_storm[var].dtype=='float32') or (ds_storm[var].dtype=='int16')):
        ds_storm[var]=ds_storm[var].astype(str)

#calculate max_wind for each storm, as an easy way to classify them
ds_storm['max_wind']=ds_storm.usa_wind.max(dim='date_time',skipna=True)
ds_storm['max_cat']=ds_storm.usa_sshs.max(dim='date_time',skipna=True)
ds_storm['yr']=ds_storm.time[:,0].dt.year
ds_storm['mon']=ds_storm.time[:,0].dt.month

ds_storm

In [ ]:
#list the storms and dates for west pacific year
sub = ds_storm.where((ds_storm.basin=='WP') & (ds_storm.max_cat>=3) & (ds_storm.season==2017),drop=True)
for i in range(len(sub.name)):
    print(sub.time[i,0].data,sub.name[i,0].data,sub.max_cat[i,0].data)
#list the storms and dates for west pacific year
sub = ds_storm.where((ds_storm.basin=='WP') & (ds_storm.max_cat>=3) & (ds_storm.season==2018),drop=True)
for i in range(len(sub.name)):
    print(sub.time[i,0].data,sub.name[i,0].data,sub.max_cat[i,0].data)
#list the storms and dates for west pacific year
sub = ds_storm.where((ds_storm.basin=='WP') & (ds_storm.max_cat>=3) & (ds_storm.season==2019),drop=True)
for i in range(len(sub.name)):
    print(sub.time[i,0].data,sub.name[i,0].data,sub.max_cat[i,0].data)
#list the storms and dates for west pacific year
sub = ds_storm.where((ds_storm.basin=='WP') & (ds_storm.max_cat>=3) & (ds_storm.season==2020),drop=True)
for i in range(len(sub.name)):
    print(sub.time[i,0].data,sub.name[i,0].data,sub.max_cat[i,0].data)

# read in NOAA OI SST
- calculate climatology, anomaly, monthly anomaly

In [ ]:
storm_list,ilist = ['BAVI','MAYSAK','HAISHEN'],[]
for name in storm_list:
    iloc = np.where((ds_storm.name==name) & (ds_storm.time>np.datetime64('2020-07-01')))[0][0]
    print(name,iloc)
    ilist.append(iloc)

In [ ]:
%%time

adir = 'F:/data/sat_data/sst/noaa_oisst/www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/'
dir_pattern_zarr = adir + 'avhrr_zarr2/'
ds_sst = xr.open_zarr(dir_pattern_zarr,consolidated=True)

sst_climatology = ds_sst.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)

sst_anomaly = ds_sst.groupby('time.dayofyear')-sst_climatology

sst_anomaly_monthly = sst_anomaly.resample(time='1MS').mean(keep_attrs=True,skipna=False)

ds_sst

# collocate sst and sst_anomaly with the larger storms
- make a subset of storms after 1982 in august cat 3 or larger
- only include storms on west side

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
for i in ilist:
    ax.plot(ds_storm.lon[i,:],ds_storm.lat[i,:], transform=ccrs.PlateCarree())
ax.set_extent([110, 150, 10, 60], crs=ccrs.PlateCarree())
ax.coastlines('50m')
ax.stock_img()

In [ ]:
subset_storms = ds_storm.isel(storm=ilist)
#subset_storms

# where do big storms start out?  only look at 1980 to present (data goes back to 1945)

In [ ]:
ds_storm['yr']=ds_storm.time[:,0].dt.year
big = ds_storm.where((ds_storm.max_wind>80) & (ds_storm.yr>1980),drop=True)
#big

# plot the first point the typhoon is classified as a storm, color is max_wnd

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
for i in ilist:
    plt.scatter(big.lon[:,0],big.lat[:,0],c=big.max_wind,transform=ccrs.PlateCarree())
ax.set_extent([110, 200, 0, 50], crs=ccrs.PlateCarree())
ax.coastlines('50m')
ax.stock_img()
plt.colorbar(label='max winds')

In [ ]:
plt.hist(big.time[:,0].dt.year,bins=np.arange(1980,2022))
plt.grid()

In [ ]:
ds_storm['yr']=ds_storm.time[:,0].dt.year
ds_storm['mon']=ds_storm.time[:,0].dt.month
big = ds_storm.where((ds_storm.max_cat>=3) & (ds_storm.yr>1982) & (ds_storm.mon==8),drop=True)
plt.hist(big.time[:,0].dt.year,bins=np.arange(1982,2022))
plt.grid()

# monthly frequency of storms

In [ ]:
bins  = np.arange(0,13)
h = histogram(ds_storm.time[:,0].dt.month, bins=[bins])
h.plot(marker='.')
plt.grid()


# calculate histogram of number of storms per year

In [ ]:
bins = np.arange(1982,2022)+.5
h = histogram(big.time[:,0].dt.year, bins=[bins])
h.plot(marker='.')

In [ ]:
%%time
#select storms august
big2 = ds_storm.where((ds_storm.max_cat>=3) & (ds_storm.yr>1982) & (ds_storm.mon==8) & (ds_storm.lon.max('date_time')<260),drop=True)
big2['sst_anom']=sst_anomaly.sst.interp(lat=big2.lat,lon=big2.lon,time=big2.time).compute()
big2['sst']=ds_sst.sst.interp(lat=big2.lat,lon=big2.lon,time=big2.time).compute()

#select west pacific august data
wp = sst_anomaly_monthly.sel(lon=slice(100,175),lat=slice(0,45))
aug = wp.sst[11::12,:,:]

# read in NOAA OI SST, find the storm data for 2020 storms that were tied to the CA wildfires 

In [6]:
storm_list,ilist = ['BAVI','MAYSAK','HAISHEN'],[]
for name in storm_list:
    iloc = np.where((ds_storm.name==name) & (ds_storm.time>np.datetime64('2020-07-01')))[0][0]
    print(name,iloc)
    ilist.append(iloc)

# plot the storm tracks

In [7]:
tem = big2 #ds_storm.where((ds_storm.max_wind>80) & (ds_storm.yr>1980),drop=True)
fig,ax = plt.subplots(2,figsize=(12,6))
ax[0].hist(tem.lat[:,0],bins=np.arange(1.5,24.5))
ax[1].hist(tem.lon[:,0],bins=np.arange(110.5,180.5))

# is there a trend in latitude start?
- yes, 1.3 deg lat per decade

In [13]:
big = ds_storm.where((ds_storm.max_cat>=3) & (ds_storm.yr>1982) & (ds_storm.mon==8),drop=True)
plt.scatter(big.yr,big.lat[:,0])
m, b = np.polyfit(big.yr,big.lat[:,0], 1)
x=np.arange(1983,2020)
plt.plot(x, m*x + b)
print(m,b)

In [14]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
for i in range(len(big2.storm)):
    if big2.yr[i]>=2015:
        continue
    #ax.scatter(big2.lon[i,:],big2.lat[i,:],c=big2.sst_anom[i,:],vmin=-2,vmax=2,cmap='RdBu_r',transform=ccrs.PlateCarree(),label=big2.name[i].data,linewidth=3)
    ax.scatter(big2.lon[i,:],big2.lat[i,:],c=big2.sst_anom[i,:],s=big2.usa_wind[i,:]/2,vmin=-2,vmax=2,cmap='RdBu_r',transform=ccrs.PlateCarree(),linewidth=3)
ax.set_extent([100,240, 0, 55], crs=ccrs.PlateCarree())
ax.coastlines('50m')
ax.stock_img()
ax.legend()
r1,r2,r3,r4 = 120,147,15,25
ax.plot([r1,r1],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r2,r2],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r3,r3],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r4,r4],'k', transform=ccrs.PlateCarree())
r1,r2,r3,r4 = 127,182,5,20
ax.plot([r1,r1],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r2,r2],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r3,r3],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r4,r4],'k', transform=ccrs.PlateCarree())

# storms after 2015

In [15]:
#plt.scatter(big2.lon,big2.lat,c=big2.sst_anom.mean('date_time'))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
for i in range(len(big2.storm)):
    if big2.yr[i]<2015:
        continue
    #ax.scatter(big2.lon[i,:],big2.lat[i,:],c=big2.sst_anom[i,:],vmin=-2,vmax=2,cmap='RdBu_r',transform=ccrs.PlateCarree(),label=big2.name[i].data,linewidth=3)
    ax.scatter(big2.lon[i,:],big2.lat[i,:],c=big2.sst_anom[i,:],s=big2.usa_wind[i,:]/2,vmin=-2,vmax=2,cmap='RdBu_r',transform=ccrs.PlateCarree(),linewidth=3)
ax.set_extent([100,240, 0, 55], crs=ccrs.PlateCarree())
ax.coastlines('50m')
ax.stock_img()
ax.legend()
r1,r2,r3,r4 = 120,147,15,25
ax.plot([r1,r1],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r2,r2],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r3,r3],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r4,r4],'k', transform=ccrs.PlateCarree())
r1,r2,r3,r4 = 127,182,5,20
ax.plot([r1,r1],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r2,r2],[r3,r4],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r3,r3],'k', transform=ccrs.PlateCarree())
ax.plot([r1,r2],[r4,r4],'k', transform=ccrs.PlateCarree())

# looking at cat 3 storm frequency in August

- since 2015 consistently more storms

In [16]:
plt.hist(big.time[:,0].dt.year,bins=np.arange(1982,2022))
plt.grid()

# What does the SST data look like in the regions where most storms originate?

In [17]:
fig = plt.figure(figsize=(15,6))
ts = sst_anomaly_monthly.sst[11::12,:,:].sel(lon=slice(r1,r2),lat=slice(r3,r4)).mean({'lat','lon'})
plt.plot(ts.time,ts,linewidth=2,color='b')
plt.axhline(0,color='k')
plt.xlabel('Year')
plt.ylabel('SST Anomaly (K)')
plt.savefig('F:/data/project_data/fluxsat/korea/aug_sst_max_north_area_large.png')

# what does august look like for the region where the 2020 storms started?

In [19]:
fig = plt.figure(figsize=(15,6))
ts = sst_anomaly_monthly.sst[11::12,:,:].sel(lon=slice(s1,s2),lat=slice(s3,s4)).mean({'lat','lon'})
plt.plot(ts.time,ts,linewidth=2,color='b')
plt.axhline(0,color='k')
plt.xlabel('Year')
plt.ylabel('SST Anomaly (K)')
plt.savefig('F:/data/project_data/fluxsat/korea/aug_sst_max_north_area.png')

# make west pacific subset and then save only august

In [20]:
#wp = sst_anomaly_monthly.sel(lon=slice(100,175),lat=slice(0,45))
#aug = wp.sst[11::12,:,:]

In [27]:
tem = aug.sel(time=slice('2000','2021'))
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1,
                     projection=ccrs.PlateCarree(central_longitude=180))
ax.set_extent([100,175, 0, 45], crs=ccrs.PlateCarree())
ax.coastlines('50m')
ax.stock_img()
cax = tem[0,:,:].plot(ax=ax,
    add_colorbar=True,
    cmap='coolwarm',
    vmin=-2, vmax=2,transform=ccrs.PlateCarree(),
    cbar_kwargs={
        'extend':'neither'
    }
)
def animate(frame):
    cax.set_array(tem[frame,:,:].values.flatten())
    ax.set_title("Time = " + str(tem.coords['time'].values[frame])[:13])
    ax.coastlines('50m')
    ax.stock_img()


# Finally, we use the animation module to create the animation.
ani = animation.FuncAnimation(
    fig,             # figure
    animate,         # name of the function above
    frames=21,       # Could also be iterable or list
    interval=200     # ms between frames
)

In [28]:
HTML(ani.to_jshtml())

In [29]:
ani.save('F:/data/project_data/fluxsat/korea/basic_animation.mp4', fps=1, extra_args=['-vcodec', 'libx264'])


# load SST monthly anomaly for august
- this is a small enough dataset that to make everything faster we can just load into memory and avoid chuncking issues

In [30]:
sst = sst_anomaly_monthly.anom[11::12,:,:].load()  #AUGUST

# remove mean and trend
- detrend the data and put back into xr.DataArray & fill back in nan
- sst2 = sst_debias - sst_slope.slope*sst_debias


# plot SST trend

In [31]:
sst_debias = sst - sst.mean({'time'})
sst2 = signal.detrend(sst_debias.fillna(0),axis=0,type='linear')
sst2 = xr.DataArray(sst2,dims=['time','lat','lon'],coords={'time':sst_debias.time,'lat':sst_debias.lat,'lon':sst_debias.lon})
sst2 = sst2.where(~np.isnan(sst))
dif = sst_debias-sst2 #cal difference between sst with bias removed and sst with trend removed
((dif.sel(time='2010-08-01')-dif.sel(time='2000-08-01'))).plot(vmin=-1,vmax=1,cmap='RdBu_r',cbar_kwargs={'label': 'SST trend (K/decade)'})

In [32]:
plt.plot(sst2.sel(lon=150,lat=40,method='nearest'),label='detrend')
plt.plot(sst_debias.sel(lon=150,lat=40,method='nearest'),label='debias')
plt.legend()

In [33]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
coslat = np.cos(np.deg2rad(sst2.coords['lat'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(sst2, weights=wgts)

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
eof1 = solver.eofsAsCorrelation(neofs=5)
pc1 = solver.pcs(npcs=5, pcscaling=1)

# Plot the leading EOF expressed as correlation in the Pacific domain.
clevs = np.linspace(-1, 1, 11)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
fill = eof1[0].plot.contourf(ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='w', edgecolor='k')
cb = plt.colorbar(fill, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF1 (ENSO - M1) expressed as correlation', fontsize=16)

# Plot the leading PC time series.
plt.figure()
pc1[:, 0].plot(color='b', linewidth=2)
ax = plt.gca()
ax.axhline(0, color='k')
ax.set_ylim(-3, 3)
ax.set_xlabel('Year')
ax.set_ylabel('Normalized Units')
ax.set_title('PC1 Time Series', fontsize=16)


In [34]:
# Plot the leading EOF expressed as correlation in the Pacific domain.
clevs = np.linspace(-1, 1, 11)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
fill = eof1[1].plot.contourf(ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='w', edgecolor='k')
cb = plt.colorbar(fill, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF2 expressed as correlation', fontsize=16)

# Plot the leading PC time series.
plt.figure()
pc1[:, 1].plot(color='b', linewidth=2)
ax = plt.gca()
ax.axhline(0, color='k')
ax.set_ylim(-3, 3)
ax.set_xlabel('Year')
ax.set_ylabel('Normalized Units')
ax.set_title('PC2 Time Series', fontsize=16)

In [35]:
# Plot the leading EOF expressed as correlation in the Pacific domain.
clevs = np.linspace(-1, 1, 11)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
fill = eof1[2].plot.contourf(ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='w', edgecolor='k')
cb = plt.colorbar(fill, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF3 ENSO expressed as correlation', fontsize=16)

# Plot the leading PC time series.
plt.figure()
pc1[:, 2].plot(color='b', linewidth=2)
ax = plt.gca()
ax.axhline(0, color='k')
ax.set_ylim(-3, 3)
ax.set_xlabel('Year')
ax.set_ylabel('Normalized Units')
ax.set_title('PC3 Time Series', fontsize=16)

In [36]:
# Plot the leading EOF expressed as correlation in the Pacific domain.
clevs = np.linspace(-1, 1, 11)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
fill = eof1[3].plot.contourf(ax=ax, levels=clevs, cmap=plt.cm.RdBu_r,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='w', edgecolor='k')
cb = plt.colorbar(fill, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF4 expressed as correlation', fontsize=16)

# Plot the leading PC time series.
plt.figure()
pc1[:, 3].plot(color='b', linewidth=2)
ax = plt.gca()
ax.axhline(0, color='k')
ax.set_ylim(-3, 3)
ax.set_xlabel('Year')
ax.set_ylabel('Normalized Units')
ax.set_title('PC4 Time Series', fontsize=16)

# reconstruct data

In [37]:
reconstructed_data0 = solver.reconstructedField(0)
reconstructed_data1 = solver.reconstructedField(1)
reconstructed_data2 = solver.reconstructedField(2)
reconstructed_data3 = solver.reconstructedField(3)
reconstructed_data4 = solver.reconstructedField(4)

In [38]:
#r1,r2,r3,r4 = 120,145,15,25
fig = plt.figure(figsize=(15,6))
#ts = sst_anomaly_monthly.sst[11::12,:,:].sel(lon=slice(130,150),lat=slice(20,30)).mean({'lat','lon'})
ts = sst_anomaly_monthly.sst[11::12,:,:].sel(lon=slice(r1,r2),lat=slice(r3,r4)).mean({'lat','lon'})
plt.plot(ts.time,ts,linewidth=2,color='b',label='data')
ts0 = reconstructed_data0.sel(lon=slice(r1,r2),lat=slice(r3,r4)).mean({'lat','lon'})
ts1 = reconstructed_data1.sel(lon=slice(r1,r2),lat=slice(r3,r4)).mean({'lat','lon'})
ts2 = reconstructed_data2.sel(lon=slice(r1,r2),lat=slice(r3,r4)).mean({'lat','lon'})
ts3 = reconstructed_data3.sel(lon=slice(r1,r2),lat=slice(r3,r4)).mean({'lat','lon'})
ts4 = reconstructed_data4.sel(lon=slice(r1,r2),lat=slice(r3,r4)).mean({'lat','lon'})
ts_trend = dif.sel(lon=slice(130,150),lat=slice(20,30)).mean({'lat','lon'})
plt.plot(ts.time,ts0+ts_trend,linewidth=2,color='r',label='reconstructed0')
plt.plot(ts.time,ts1+ts_trend,linewidth=2,color='m',label='reconstructed1')
plt.plot(ts.time,ts2+ts_trend,linewidth=2,color='c',label='reconstructed2')
plt.plot(ts.time,ts3+ts_trend,linewidth=2,color='g',label='reconstructed3')
plt.plot(ts.time,ts4+ts_trend,linewidth=2,color='k',label='reconstructed4')
plt.xlabel('Year')
plt.ylabel('SST Anomaly (K)')
plt.legend()
plt.savefig('F:/data/project_data/fluxsat/korea/aug_sst_reconstructed.png')

In [39]:
plt.plot(ts.time,ts0+ts_trend,linewidth=2,color='r',label='reconstructed1')
plt.plot(ts.time,ts1+ts_trend,linewidth=2,color='m',label='reconstructed1')
plt.plot(ts.time,ts2+ts_trend,linewidth=2,color='c',label='reconstructed1')
plt.plot(ts.time,ts3+ts_trend,linewidth=2,color='g',label='reconstructed1')


# what does buoy data look like?

In [40]:
fname='https://dods.ndbc.noaa.gov/thredds/dodsC/data/stdmet/52211/52211.ncml'
ds_buoy = xr.open_dataset(fname).rename({'latitude':'lat','longitude':'lon'})
ds_buoy['mon']=ds_buoy.time.dt.month
#ds_buoy

In [41]:
ds_buoy_month = ds_buoy.resample(time='1MS').mean(keep_attrs=True,skipna=True)
ds_buoy_month.sea_surface_temperature.plot()
buoy_aug = ds_buoy_month.where(ds_buoy_month.mon==8,drop=True)
buoy_aug.sea_surface_temperature.plot()

In [42]:
#too slow
buoy_clim = sst_climatology.sst.sel(lat=ds_buoy.lat,lon=ds_buoy.lon,method='nearest')
sst_anomaly_buoy = ds_buoy.sea_surface_temperature.groupby('time.dayofyear')-buoy_clim[:,0,0]
sst_anomaly_buoy_monthly = sst_anomaly_buoy.resample(time='1MS').mean(keep_attrs=True,skipna=True)
sst_anomaly_buoy_monthly[10::12].plot()
plt.ylabel('SST anomaly (K)')
print(sst_anomaly_buoy_monthly[10::12].load().data[-1])

# plot NOAA OI SST, reconstructed SST, and buoy SST all at buoy SST location
- Just a note - if they don't agree there is something probably wrong because NOAA OI SSTs uses in situ data in their analysis

In [43]:
fig = plt.figure(figsize=(15,6))
ts = sst_anomaly_monthly.sst[11::12,:,:].sel(lat=ds_buoy.lat,lon=ds_buoy.lon,method='nearest')
plt.plot(ts.time,ts[:,0,0],linewidth=2,color='b',label='data')
ts = reconstructed_data4.sel(lat=ds_buoy.lat,lon=ds_buoy.lon,method='nearest')
ts_trend = dif.sel(lat=ds_buoy.lat,lon=ds_buoy.lon,method='nearest')
plt.plot(ts.time,ts[:,0,0]+ts_trend[:,0,0],linewidth=2,color='r',label='reconstructed')
sst_anomaly_buoy_monthly[10::12].plot(color='k',marker='.',label='buoy')
plt.xlabel('Year')
plt.ylabel('SST Anomaly (K)')
plt.legend()
plt.savefig('F:/data/project_data/fluxsat/korea/aug_sst_reconstructed_buoy.png')

# so what is going on? are all months warming like this?

In [44]:
for imon in range(12):
    imon2 = imon+4
    sst = sst_anomaly_monthly.anom[imon2::12,:,:].load()  #monthly
    sst_debias = sst - sst.mean({'time'})
    sst2 = signal.detrend(sst_debias.fillna(0),axis=0,type='linear')
    sst2 = xr.DataArray(sst2,dims=['time','lat','lon'],coords={'time':sst_debias.time,'lat':sst_debias.lat,'lon':sst_debias.lon})
    sst2 = sst2.where(~np.isnan(sst))
    dif = sst_debias-sst2 #cal difference between sst with bias removed and sst with trend removed
    time0,time1='2010-'+str(imon+1).zfill(2)+'-01','2000-'+str(imon+1).zfill(2)+'-01'
    ((dif.sel(time=time0)-dif.sel(time=time1))).plot(vmin=-1,vmax=1,cmap='RdBu_r',cbar_kwargs={'label': 'SST trend (K/decade)'})
    plt.savefig('F:/data/project_data/fluxsat/korea/sst_trend_mon'+str(imon+1)+'.png')
    plt.clf()
    

# when are SSTs warm enough for TCs?
- TC require SST > 26.5  ref: https://journals.ametsoc.org/view/journals/clim/28/20/jcli-d-14-00637.1.xml
- so write a function that calculates fraction of days in a month that SSTs >26.5
- then once that is working, caluclate it for each year
- then calculate if there is a trend in # days

In [45]:
def fracdy(x):
    return (((x.where(x>26.5)/x).sum({'time'}))/x.sizes['time'])
def numdy(x):
    return ((x.where(x>26.5)/x).sum({'time'}))
frac = ds_sst.sst.groupby('time.month').map(fracdy)
num = ds_sst.sst.groupby('time.month').map(numdy)

In [46]:
frac_yr = []
for lyr in range(1982,2021):
    tem = ds_sst.sst.sel(time=str(lyr))
    frac = tem.groupby('time.month').map(fracdy)
    frac_yr.append(frac)
frac_yr = xr.concat(frac_yr, dim='time')
frac_yr['time']=np.arange(1982,2021)

In [47]:
#r1,r2,r3,r4 = 120,145,15,25
ts =frac_yr.isel(month=6).sel(lon=r2,lat=r4,method='nearest')
f = (ts.sel(time=slice(2015,2020)).mean()-ts.sel(time=slice(1985,1990)).mean()).load()
print(f)

In [48]:
ts =frac_yr.isel(month=7)
f = (ts.sel(time=slice(2015,2020)).mean('time')-ts.sel(time=slice(1985,1990)).mean('time')).load()
fig = plt.figure(figsize=(15,6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
f.plot(vmin=-.75,vmax=.75,cmap='RdBu_r',transform=ccrs.PlateCarree())
#ax.set_extent([110, 150, 10, 60], crs=ccrs.PlateCarree())
ax.coastlines('50m')
ax.stock_img()

In [49]:
ts.sel(lat=25,lon=145,method='nearest').plot()

In [50]:
ts_debias = ts - ts.mean({'time'})
ts2 = signal.detrend(ts_debias.fillna(0),axis=0,type='linear')

In [51]:
time0,time1=2010,2000
dif = ts_debias-ts2 #cal difference between sst with bias removed and sst with trend removed
f = ((dif.sel(time=time0)-dif.sel(time=time1))) #.plot(vmin=-.25,vmax=.25,cmap='RdBu_r',cbar_kwargs={'label': 'SST trend (K/decade)'})
fig = plt.figure(figsize=(15,6))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
f.plot(vmin=-.25,vmax=.25,cmap='RdBu_r',transform=ccrs.PlateCarree())
#ax.set_extent([110, 150, 10, 60], crs=ccrs.PlateCarree())
ax.coastlines('50m')
ax.stock_img()

In [52]:
big

In [53]:
sub = ds_storm.where((ds_storm.max_wind>50) & (ds_storm.yr>1980))
plt.scatter(sub.lon[:,0],sub.lat[:,0],c=sub.yr)
plt.colorbar()

In [ ]:
hh=[]
for iyr in range(1980,2021):
    sub = ds_storm.where((ds_storm.max_wind>50) & (ds_storm.yr==1980),drop=True)
    h = plt.hist(sub.lat[:,0],bins=np.arange(0,40,2))
    h1 = xr.DataArray(h[0]/sum(h[0]),coords={'lat':h[1][:-1]},dims=('lat'))
    hh.append(h1)

In [ ]:
h2 = xr.concat(hh,dim='year')

In [ ]:
plt.plot(h2[:,0])

In [ ]:
h

In [ ]:
# can i get average sst at the locations the storms went over their first 5-10 days?
big = ds_storm.where((ds_storm.max_cat>=3) & (ds_storm.yr>1982) & (ds_storm.mon==8),drop=True)
big

In [ ]:
#big2 is for all storms that START in august
#sub2 is for all storms that have any data in august
ds_storm['yr']=ds_storm.time[:,0].dt.year
ds_storm['mon']=ds_storm.time[:,0].dt.month
#subset to cat 3 & post 1982
big = ds_storm.where((ds_storm.max_cat>=3) & (ds_storm.yr>1982),drop=True)
#big2 = ds_storm.where((ds_storm.max_cat>=3) & (ds_storm.yr>1982) & (ds_storm.mon==8),drop=True)
#big2a = ds_storm.where((ds_storm.max_cat>=3) & (ds_storm.yr>1982) & (ds_storm.mon==8) & (ds_storm.lon.max('date_time')<260) & (ds_storm.lat.max('date_time')>25),drop=True)


# Plot SST anomalies along storm track

In [ ]:
plt.scatter(big2.yr,big2.sst_anom.mean('date_time'),c=big2.max_cat)
plt.colorbar()

# Plot SSTs to check 26.5 threshold point. only use mean sst for first 20 obs.

In [ ]:
plt.scatter(big2.yr,big2.sst.isel(date_time=slice(0,20)).mean('date_time'),c=big2.max_cat)
plt.colorbar()

In [ ]:
#for i in range(63):
#    tem=np.sum(np.isfinite(np.where(big2.wmo_wind[i,:]>10)))
#    plt.scatter(big2.yr[i],tem)


In [ ]:
plt.scatter(big2.yr,big2.numobs,c=big2.max_cat)
plt.colorbar()

In [ ]:
plt.scatter(big2.sst,big2.wmo_wind)
plt.xlabel('SST ($\circ$C)')
plt.ylabel('Wind Speed (kts)')
plt.colorbar()